# Titanic Challenge

In [1]:
import numpy as np
# !pip install plotly
import plotly.express as px

In [2]:
import pandas as pd
train = pd.read_csv('train.csv')
train.head()

,Id,Outcome,Pclass,Name,Sex,Age,Sibsp,Parch,Fare
0,1,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,2,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,3,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,4,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,5,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


Now let's bin and onehot

In [3]:
agebins = pd.cut(train.Age,bins=[0,6,13,19,60,999],labels=['infant','child','teen','adult','senior'])
train['agebins'] = agebins
train = pd.get_dummies(train,columns=['Sex','Pclass','agebins'])
train.head()

,Id,Outcome,Name,Age,Sibsp,Parch,Fare,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,agebins_infant,agebins_child,agebins_teen,agebins_adult,agebins_senior
0,1,0,Mr. Owen Harris Braund,22.0,1,0,7.2500,0,1,0,0,1,0,0,0,1,0
1,2,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,38.0,1,0,71.2833,1,0,1,0,0,0,0,0,1,0
2,3,1,Miss. Laina Heikkinen,26.0,0,0,7.9250,1,0,0,0,1,0,0,0,1,0
3,4,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,35.0,1,0,53.1000,1,0,1,0,0,0,0,0,1,0
4,5,0,Mr. William Henry Allen,35.0,0,0,8.0500,0,1,0,0,1,0,0,0,1,0


In [4]:
train[train.Fare == 0].shape
# fig1 = px.pie(survivors, title=f'Class of {survivors.shape[0]} survivors', values="Pclass",names="Pclass")
# fig2 = px.pie(sinkers, title=f'Class of {sinkers.shape[0]} sinkers', values="Pclass",names="Pclass")
#fig2 = px.histogram(train, x="Fare")
#fig2.show()

(11, 17)

In [5]:
train['LogFare'] = np.log(train['Fare'].replace(0, 0.01).replace(np.NaN, 0.01))
train.head()

,Id,Outcome,Name,Age,Sibsp,Parch,Fare,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,agebins_infant,agebins_child,agebins_teen,agebins_adult,agebins_senior,LogFare
0,1,0,Mr. Owen Harris Braund,22.0,1,0,7.2500,0,1,0,0,1,0,0,0,1,0,1.981001
1,2,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,38.0,1,0,71.2833,1,0,1,0,0,0,0,0,1,0,4.266662
2,3,1,Miss. Laina Heikkinen,26.0,0,0,7.9250,1,0,0,0,1,0,0,0,1,0,2.070022
3,4,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,35.0,1,0,53.1000,1,0,1,0,0,0,0,0,1,0,3.972177
4,5,0,Mr. William Henry Allen,35.0,0,0,8.0500,0,1,0,0,1,0,0,0,1,0,2.085672


In [9]:
train['NotAlone'] = train['Sibsp'] + train['Parch']
print(train[train.NotAlone > 0].to_string())
list(train.columns.values)

      Id  Outcome                                                                               Name    Age  Sibsp  Parch      Fare  Sex_female  Sex_male  Pclass_1  Pclass_2  Pclass_3  agebins_infant  agebins_child  agebins_teen  agebins_adult  agebins_senior   LogFare  NotAlone
0      1        0                                                             Mr. Owen Harris Braund  22.00      1      0    7.2500           0         1         0         0         1               0              0             0              1               0  1.981001         1
1      2        1                                 Mrs. John Bradley (Florence Briggs Thayer) Cumings  38.00      1      0   71.2833           1         0         1         0         0               0              0             0              1               0  4.266662         1
3      4        1                                        Mrs. Jacques Heath (Lily May Peel) Futrelle  35.00      1      0   53.1000           1         0       

['Id',
 'Outcome',
 'Name',
 'Age',
 'Sibsp',
 'Parch',
 'Fare',
 'Sex_female',
 'Sex_male',
 'Pclass_1',
 'Pclass_2',
 'Pclass_3',
 'agebins_infant',
 'agebins_child',
 'agebins_teen',
 'agebins_adult',
 'agebins_senior',
 'LogFare',
 'NotAlone']

Now let's start getting ready to model

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train[["Id", "Sex_female", "Sex_male",
                                                            "Pclass_1", "Pclass_2", "Pclass_3",
                                                            "agebins_infant","agebins_child",
                                                            "agebins_teen","agebins_adult",
                                                            "agebins_senior",
                                                            "LogFare", "NotAlone"]],
                                                    train['Outcome'], test_size=0.2, random_state=22)

In [20]:
X_train.shape
X_train.head()

,Id,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,agebins_infant,agebins_child,agebins_teen,agebins_adult,agebins_senior,LogFare,NotAlone
440,441,1,0,0,1,0,0,0,0,1,0,2.564949,0
194,195,1,0,1,0,0,0,0,0,1,0,4.987167,0
215,216,1,0,0,0,1,0,0,0,1,0,2.070022,0
2,3,1,0,0,0,1,0,0,0,1,0,2.070022,0
75,76,0,1,0,0,1,0,0,0,1,0,2.066331,0


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential(
    [
        tf.keras.Input(shape=(400,)),    #specify input size
        ### START CODE HERE ###
        tf.keras.layers.Dense(25, activation="relu"),
        tf.keras.layers.Dense(15, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid")
        ### END CODE HERE ###
    ], name = "my_model"
)

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    X_train,y_train,
    epochs=20
)